In [2]:
from bs4 import BeautifulSoup
import requests
import pickle
import json
import pandas as pd

In [3]:
# Get all local authority codes and corresponding council names
page = requests.get("https://epc.opendatacommunities.org/docs/api/domestic#domestic-local-authority").text
soup = BeautifulSoup(page)

table = soup.findAll('table')[3]
la_code_dict = {}

for tr in table.findAll('tr')[1:]:
   code, local_auth = tr.findAll('td')
   la_code_dict[local_auth.text] = code.text

# Get local authority codes for councils in WMCA
WMCA_councils = open("../data/raw_data/WMCA_council.txt").read().split(",")[:-1]
WMCA_code = [la_code_dict[i] for i in WMCA_councils]
WMCA = dict(zip(WMCA_code, WMCA_councils))

In [4]:
# Save codes for future use
with open('../data/raw_data/WMCA_council_code.pkl', 'wb') as f:
    pickle.dump(WMCA, f)

In [5]:
# Make private
AUTH_TOKEN = "bGktbGlhbi5hbmdAd2Fyd2ljay5hYy51azpkNzBlYTliMTExMjUzODc0NDM0MTFiMGE5MDVjNDFiYmY2MTA3NmNj"

In [6]:
def get_epc_data(postcode, num_rows=100):
    """
    Pull data from Domestic Energy Performance Certificates API.

    Input:
    postcode(str): (1) Postcode 
    num_rows(int): Number of rows to pull. Max 5000 allowed at one time

    Output:
    (str): Pulled data from API

    """
    headers = {
        'Authorization': f'Basic {AUTH_TOKEN}',
        'Accept': 'application/json'
    }
    params = {
        'postcode': postcode,
        'size': num_rows
    }
    url = f'https://epc.opendatacommunities.org/api/v1/domestic/search'
    res = requests.get(url, headers=headers, params=params)
    return res.text

In [7]:
# Pull WMCA local authority data and save as CSV
result = list()

for code, council in WMCA.items():
    result.extend(json.loads(get_local_authority_data(code))['rows'])

df = pd.DataFrame(result)
df.to_csv("../data/raw_data/EPC_data.csv", index=False)